In [1]:
import random
import matplotlib.pyplot as plt

from collections import deque

import pennylane as qml
from pennylane.templates import ApproxTimeEvolution
from pennylane import numpy as np
import itertools


In [2]:
class TSP:
    def __init__(self, number_of_cities, coords_range=(0, 10000)):
        self.number_of_cities = number_of_cities
        self.coords_range = coords_range
        self.cities_coords = self.get_cities()
        self.distance_matrix = self.calculate_distance_matrix()
        self.normalized_distance_matrix = self.normalize_distance_matrix()
    
    def get_cities(self):
        cities_coords = np.random.randint(self.coords_range[0], self.coords_range[1], size = (self.number_of_cities, 2))
        return cities_coords
           
    def normalize_cities(self):
        max_coords = np.amax(self.cities_coords, axis=0)
        normalized_cities_coords = np.divide(self.cities_coords, max_coords)
        return normalized_cities_coords

    def calculate_distance_between_points(self, point_A, point_B):
        return np.sqrt((point_A[0] - point_B[0])**2 + (point_A[1] - point_B[1])**2)
    
    def calculate_distance_matrix(self):
        distance_matrix = np.zeros((self.number_of_cities, self.number_of_cities))
        for i in range(self.number_of_cities):
            for j in range(i, self.number_of_cities):
                distance_matrix[i][j] = self.calculate_distance_between_points(self.cities_coords[i], self.cities_coords[j])
                distance_matrix[j][i] = distance_matrix[i][j]
        return distance_matrix 
    
    def normalize_distance_matrix(self):
        return np.divide(self.distance_matrix, np.max(self.distance_matrix))


In [3]:
class QAOA_TSP:
    def __init__(self, tsp_instance, A_1=4, A_2=4, B=1):
        self.tsp_instance = tsp_instance
        self.weights = {'cost_weight': B, 
                        'constraint_each_visited': A_1, 
                        'constraint_each_visited_once': A_2}
        self.cost_operator = self.create_cost_operator()
    
    def calc_bit(self, i, t):
        return i + t * self.tsp_instance.number_of_cities

    def x(self, i, t):
        wire = self.calc_bit(i, t)
        return qml.Hamiltonian([0.5, -0.5], [qml.Identity(wire), qml.PauliZ(wire)])

    def create_cost_operator(self):
        A_1 = self.weights['constraint_each_visited']
        A_2 = self.weights['constraint_each_visited_once']
        B = self.weights['cost_weight']
        
        cost_of_constraint_each_visited = 0    
        for i in range(self.tsp_instance.number_of_cities):
            curr = qml.Identity(0)
            for t in range(self.tsp_instance.number_of_cities):
                curr -= self.x(i, t)    
            for t1 in range(self.tsp_instance.number_of_cities):
                for t2 in range(t1 + 1, self.tsp_instance.number_of_cities):
                    curr += 2 * self.x(i, t1) @ self.x(i, t2)
            cost_of_constraint_each_visited += curr
        cost_of_constraint_each_visited_once = 0
        for t in range(self.tsp_instance.number_of_cities):
            curr = qml.Identity(0)
            for i in range(self.tsp_instance.number_of_cities):
                curr -= self.x(i, t)
            for i1 in range(self.tsp_instance.number_of_cities):
                for i2 in range(i1 + 1, self.tsp_instance.number_of_cities):
                    curr += 2 * self.x(i1, t) @ self.x(i2, t)
            cost_of_constraint_each_visited += curr
        
        cost_of_visiting_cities = 0
        for i, j in itertools.permutations(range(0, self.tsp_instance.number_of_cities), 2):
            curr = qml.Identity(0)
            for t in range(self.tsp_instance.number_of_cities):
                inc_t = t + 1
                if inc_t == self.tsp_instance.number_of_cities:
                    inc_t = 0
                curr += self.x(i, t) @ self.x(j, inc_t)
            cost_of_visiting_cities += float(self.tsp_instance.normalized_distance_matrix[i][j]) * curr 
        
        cost_operator = (
            A_1 * cost_of_constraint_each_visited + 
            A_2 * cost_of_constraint_each_visited_once +
            B * cost_of_visiting_cities
        )
                
        return cost_operator

In [4]:
def hadamard_layer(wires):
    for i in range(wires):
        qml.Hadamard(i)

In [5]:
def create_mixing_hamitonian(wires, const=1/2):
    hamiltonian = qml.Identity(0)
    for i in range(wires):
        hamiltonian += qml.Hamiltonian([const], [qml.PauliX(i)])
    # ApproxTimeEvolution(hamiltonian, time, 1)
    return hamiltonian

In [169]:
n = 3
dev = qml.device("default.qubit", wires=n**2)
tsp_instance = TSP(n)

def run_learning(A_1, A_2, B, n, dev, tsp_instance):
    layers = 6
    qaoa_tsp = QAOA_TSP(tsp_instance, A_1, A_2, B)

    def qaoa_layer(gamma, beta):
        qml.qaoa.cost_layer(gamma, qaoa_tsp.cost_operator)
        qml.qaoa.mixer_layer(beta, create_mixing_hamitonian(n**2))
    
    def circuit(params, n_layers=2):
        hadamard_layer(n**2)
        qml.layer(qaoa_layer, n_layers, params[0], params[1])    

    @qml.qnode(dev)
    def cost_function(params):
        circuit(params, n_layers=layers)
        x = qml.expval(qaoa_tsp.cost_operator)
        return x
    
    optimizer = qml.AdamOptimizer()
    steps = 70
    params = np.array([[0.5]*layers, [0.5]*layers], requires_grad=True)

    for i in range(steps):
        params = optimizer.step(cost_function, params)
    print('.', end='')

    @qml.qnode(dev)
    def probability_circuit(gamma, alpha):
        circuit([gamma, alpha], n_layers=layers)
        return qml.probs(wires=range(n**2))

    probs = probability_circuit(params[0], params[1])
    return probs

In [170]:
def get_distance(key):
    get_bin = lambda x, n: format(x, 'b').zfill(n)
    results = np.array_split(list(get_bin(key, n**2)), n)
    dist = 0
    tab = []
    for result in results:
        tab.append(list(result).index('1'))

    for i in range(0, n):
        j = i - 1
        # print(tsp_instance.normalized_distance_matrix[i][j])
        dist += tsp_instance.normalized_distance_matrix[tab[i]][tab[j]]

    return dist

def check_results(probs):
    get_bin = lambda x, n: format(x, 'b').zfill(n)
    correct_results_3 = ("100010001", "100001010", "010100001", "010001100", "001100010", "001010100")
    correct_results_4 = ("0001100001000010","0010010010000001","0100100000010010","1000000100100100","1000010000100001","0100001000011000","0001001001001000","0010000110000100","0100000110000010","0010100000010100","0001010000101000","0001100000100100","1000000101000010","1000001001000001","0100001010000001", "0100000100101000", "0010010000011000", "0100100000100001", "1000001000010100", "0001001010000100", "0001010010000010","0010000101001000", "1000010000010010", "0010100001000001")
    
    result_dict = {key: float(val) for key, val in enumerate(probs)}
    result_dict = dict(sorted(result_dict.items(), key=lambda item: item[1], reverse=True))
    score = 0
    for key, val in result_dict.items():
        if get_bin(key, n**2) not in correct_results_4:
            score += val*20
        else:
            score += val*get_distance(key)
    return score

In [153]:
#4, 4, 1 -> 9.717510535614439
probs = run_learning(5.53828981, 5.38986829, 0.2460077, n, dev, tsp_instance)

....................

In [154]:
print(check_results(probs))

9.652341434957128


In [ ]:
correct_results_4 = ("0001100001000010","0010010010000001","0100100000010010","1000000100100100","1000010000100001","0100001000011000","0001001001001000","0010000110000100","0100000110000010","0010100000010100","0001010000101000","0001100000100100","1000000101000010","1000001001000001","0100001010000001", "0100000100101000", "0010010000011000", "0100100000100001", "1000001000010100", "0001001010000100", "0001010010000010","0010000101001000", "1000010000010010", "0010100001000001")

correct_results_3 = ("100010001", "100001010", "010100001", "010001100", "001100010", "001010100")

get_bin = lambda x, n: format(x, 'b').zfill(n)
result_dict = {key: float(val) for key, val in enumerate(probs)}
result_dict = dict(sorted(result_dict.items(), key=lambda item: item[1], reverse=True))
for key, val in result_dict.items():
    print(f"Key: {get_bin(key, n**2)} with probability {val}   | correct: {'True, dinstance: '+str(get_distance(key)) if get_bin(key, n**2) in correct_results_3 else 'False'}")

In [171]:
n_ = 3
dev_ = qml.device("default.qubit", wires=n_**2)
tsp_instance_ = TSP(n_)

def run_learning_n_get_results(p):
    p = [float(x) for x in p]
    probs = run_learning(p[0], p[0], p[1], n_, dev_, tsp_instance_)
    return float(check_results(probs))
# run_learning_n_get_results((3.4, 1.2, 5.1))

def minimize():
    n_iterations=10
    print_every=1
    pop_size=50
    elite_frac=0.1

    n_elite=int(pop_size*elite_frac)

    scores_deque = deque(maxlen=100)
    scores = []
    best_weight = [4, 1]
    best_reward = 1000
    mean = [1] * len(best_weight)
    cov = np.identity(len(best_weight)) 
    for i_iteration in range(1, n_iterations+1):
        points = np.random.multivariate_normal(mean, cov, size=pop_size)
        points = np.concatenate((points, [best_weight]), axis=0)
        # ys = np.random.normal(y_mean, 0.04, size=50)
        # print(weights_pop)
        rewards = np.array([run_learning_n_get_results(list(point)) for point in points])

        elite_idxs = rewards.argsort()[:n_elite]
        elite_weights = [points[i] for i in elite_idxs]

        # print(elite_weights)
        best_weight = elite_weights[0]

        reward = run_learning_n_get_results(best_weight)
        if reward < best_reward:
            best_weight = best_weight
            best_reward = reward
        scores_deque.append(reward)
        scores.append(reward)
        mean = np.mean(elite_weights, axis=0)
        cov = np.cov(np.stack((elite_weights), axis = 1))

        if i_iteration % print_every == 0:
            # print(self.mean)
            # print(self.cov)
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))
            # print(best_weight)
            print(f'{best_weight} with reward: {best_reward}')
    return best_weight

minimize()

....................................................Episode 1	Average Score: 20.00
[0.43565774 1.16538297] with reward: 19.999999999999414
......

KeyboardInterrupt: 